In [1]:
import nbimporter
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import pandas as pd
import math

from scipy.linalg import eigh, cholesky
from scipy.stats import norm
from pylab import plot, show, axis, subplot, xlabel, ylabel, grid


import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [2]:
def graphSelectionRates(selected, candidates, protectedClass):
    selected_candidates = selected.copy()
    candidates_all = candidates.copy()
    raceIndxs = {"Caucasian": 0, "African-American": 1, "Asian": 2, "Hispanic": 3, "Other": 4 }
    sexIndxs = {"Male":0, "Female":1}
    
    if protectedClass == "race":
        features = [col for col in selected_candidates if col.startswith("race_")]
        race_selected = selected_candidates[features].idxmax(axis=1).str.replace('race_', '')
        race_all = candidates_all[features].idxmax(axis=1).str.replace('race_', '')
        
        candidates_all["Race"] = race_all
        selected_candidates["Race"] = race_selected

        numCandidates = candidates_all["Race"].value_counts()
        numQualified = selected_candidates["Race"].value_counts()

        selectionRate1 = numQualified["Caucasian"] / numCandidates["Caucasian"]
        selectionRate2 = numQualified["Asian"] / numCandidates["Asian"]
        selectionRate3 = numQualified["African-American"] / numCandidates["African-American"]
        selectionRate4 = numQualified["Hispanic"] / numCandidates["Hispanic"]
        
        print("The selection rate for " + "Caucasian" + "s: ",selectionRate1)
        print("The selection rate for " + "Asian" + "s: ", selectionRate2)
        print("The selection rate for " + "African-American" + "s: ",selectionRate3)
        print("The selection rate for " + "Hispanic" + "s: ",selectionRate4)
        
        y_pos = np.arange(4)
        performance = [selectionRate1, selectionRate2, selectionRate3, selectionRate4]
        plt.bar(y_pos, performance, align='center', alpha=0.5)
        plt.xticks(y_pos, ("Caucasian", "Asian", "African-American", "Hispanic"))
        plt.ylabel('Selection Rate')
        plt.title('Selection Rate Comparisons by Race')
        plt.show()
    elif protectedClass == "gender":
        
        features = [col for col in selected_candidates if col.startswith("gender_")]
        gender_selected = selected_candidates[features].idxmax(axis=1).str.replace('gender_', '')
        gender_all = candidates_all[features].idxmax(axis=1).str.replace('gender_', '')

        candidates_all["Gender"] = gender_all
        selected_candidates["Gender"] = gender_selected
        
        numCandidates = candidates_all["Gender"].value_counts()
        numQualified = selected_candidates["Gender"].value_counts()
        
        print(numCandidates)
        print(numQualified) 
        
        maleSelectionRate = numQualified["male"] / numCandidates["male"]
        femaleSelectionRate = numQualified["female"] / numCandidates["female"]
        print("The selection rate for " + "Male" + "s: ", maleSelectionRate)
        print("The selection rate for " + "Female" + "s: ", femaleSelectionRate)
        y_pos = np.arange(2)
        performance = [maleSelectionRate, femaleSelectionRate]
        plt.bar(y_pos, performance, align='center', alpha=0.5)
        plt.xticks(y_pos, ("Male", "Female"))
        plt.ylabel('Selection Rate')
        plt.title('Selection Rate Comparisons by Sex')
        plt.show()
    else:
        print("Try another protected type - gender or race")
        
        
def getOnehottedDataset(dataset, features, prefixes):
# race, sex, sports, birth origin, education, educational prestige, criminal record, arrest record,
    onehotted_data = dataset.copy()
    onehotted_data = pd.get_dummies(onehotted_data, columns=features, prefix=prefixes)
    return onehotted_data

        
def classificationSelection(candidates, labels, acceptValue):
    df = candidates.copy()
    df["Labels"] = labels
    candidatesSelected = df[df["Labels"].isin([acceptValue])]
    return candidatesSelected

def transformDatapointsToScale(candidates, selectedFeatures):
    df = candidates.copy()
    for feature in selectedFeatures:
        maximum = df[feature].max()
#         print(maximum)
        df[feature] = df[feature].apply(lambda x: x / maximum)
    return df

In [5]:
# # employees = pd.read_csv("./employeeData_male_African-American.csv")
# dfs = [pd.read_csv("./employeeData_male_African-American.csv"),
#        pd.read_csv("./employeeData_male_Caucasian.csv"),
#       ]
# #        pd.read_csv("./employeeData_female_African-American.csv")]
# employees = pd.concat(dfs)
employees = pd.read_csv("employees_test.csv")

In [7]:
# One-hotting the dataset for use
def createLabel(dataset, metric, percentiles):
#     by quintiles
#     by manual cutoff
    if percentiles == "thirds":
        dataset.sort_values([metric], ascending=False)
        labels = []
        for x in range(len(dataset)):
            if x < len(dataset) / 3:
                labels.append(1)
            elif x < len(dataset) / 3 * 2:
                labels.append(0)
            else:
                labels.append(-1)
        dataset["Label"] = labels
        #  Need to shuffle because just ranked them by deal count or whatever the metric is...
        dataset = dataset.sample(frac=1).reset_index(drop=True)
    else: 
        print("Invalid percentile requested to create labels by ")
    return dataset


def discretize(vector, range):
    newRange = range
    vals = [range]
    x = True
    while(x):
        range -= 1
        if abs(range) <= newRange:
            vals.append(range)
        else:
            x=False
    roundedVals = []
    for x in vector:
        newVal = int(x)
        if abs(newVal) > newRange:
            # if greater than the newRange:
            if newVal < 0:
                newVal = -1*newRange
            else:
                newVal = 1*newRange
        roundedVals.append(newVal)
    return roundedVals

def assignLabels(employees, metric):
    employees_labelled = employees.copy()
    employees_labelled = employees.sort_values([metric], ascending=False)
    fifth = math.floor(len(employees_labelled) / 5)
    num = 2
    labels = []
    for x in range(5):
        for y in range(fifth):
            labels.append(num)
        num -= 1
    employees_labelled["Label"] = labels
    employees_labelled = employees_labelled.sample(frac=1).reset_index(drop=True)
    return employees_labelled


features = ["Race", "Gender", "Birth Origin", "Criminal Record",
           "Arrest Record", "LinkedIn Score", "Responsible Social Media Use", 
            "HireVue Score", "Undergraduate Degree", "Employee Referral"]
prefixes = ["race", "gender", "origin", "criminal", "arrest", "linkedin", 
           "socmedia", "hirevue", "degree", "referral"]
employees_race = employees["Race"]
employees_processed = getOnehottedDataset(employees, features, prefixes)
employees_processed["Race"] = employees_race
employees_processed.columns


selectedFeatures = [
#     "Technical Aptitude", "Sports", "Leadership Capability"
    "GPA", "Education", "Educational Prestige", "Years of Experience", "Soft Skills",
]

metric = "Manager's Assessment Score"


employees_processed = assignLabels(employees_processed, metric)

# employees_processed["Label"] = discretize(employees_processed[metric] / 1, 2)
print(employees_processed.groupby(["Label"]).size())


# Currently just a divide by max - normalization scheme...
# employees_scaled = transformDatapointsToScale(employees_processed, selectedFeatures)
employees_scaled = employees_processed

y = employees_scaled["Label"]
X = employees_scaled.drop(["Label"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)
X_train_selectedFeatures = X_train[selectedFeatures].copy()
X_test_selectedFeatures = X_test[selectedFeatures].copy()

def runLogRegModel(X, y):
    oneVsRest = OneVsRestClassifier(LogisticRegression(random_state = 0, max_iter=500))
    oneVsRest_fitted = oneVsRest.fit(X, y)
    return oneVsRest_fitted

model = runLogRegModel(X_train_selectedFeatures, y_train)

def getModelStats(model, X, y):
    y_pred = model.predict(X)
    vals = {}
    for x in range(len(y_pred)):
        if y_pred[x] not in vals:
            vals.update({y_pred[x]: 1})
        else: 
            vals[y_pred[x]] += 1
    print(vals)
    
    
    print("The accuracy of our model on the data is: ", accuracy_score(y_pred, y))
    print("The precision of our model on the data is: ", precision_score(y_pred, y, average='macro'))
    print("The recall of our model on the data is: ", recall_score(y_pred, y, average='macro'))
    print("The f1 score of our model on the data is: ", f1_score(y_pred, y, average='macro'))
    
getModelStats(model, X_train_selectedFeatures, y_train)


Label
-2    660
-1    660
 0    660
 1    660
 2    660
dtype: int64
{2: 559, -2: 575, 1: 500, -1: 421, 0: 420}
The accuracy of our model on the data is:  0.7846464646464646
The precision of our model on the data is:  0.7851378675142623
The recall of our model on the data is:  0.7812030163685201
The f1 score of our model on the data is:  0.7797667401785559


In [8]:
# One-hotting the dataset for use
features = ["Race", "Gender", "Birth Origin", "Criminal Record",
           "Arrest Record", "LinkedIn Score", "Responsible Social Media Use", 
            "HireVue Score", "Undergraduate Degree", "Employee Referral"]
prefixes = ["race", "gender", "origin", "criminal", "arrest", "linkedin", 
           "socmedia", "hirevue", "degree", "referral"]
employees_race = employees["Race"]
employees_processed = getOnehottedDataset(employees, features, prefixes)
employees_processed["Race"] = employees_race
employees_processed.columns


selectedFeatures = [
#     "Technical Aptitude", "Sports", "Leadership Capability"
    "GPA", "Education", "Educational Prestige", "Years of Experience", "Soft Skills",
]

metric = "Manager's Assessment Score"

employees_processed = assignLabels(employees_processed, metric)
employees_scaled = employees_processed

y = employees_scaled["Label"]
X = employees_scaled.drop(["Label"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

X_train_selectedFeatures = X_train[selectedFeatures].copy()
X_test_selectedFeatures = X_test[selectedFeatures].copy()


def rescaling(X):
#     print(X)
    scaler = MinMaxScaler()
    scaler.fit(X)
    data = scaler.transform(X)
    return data

X_scaled_train = rescaling(X_train_selectedFeatures)
# print(X_scaled_train)

neigh = KNeighborsClassifier(n_neighbors=4, metric="minkowski", p=3)
knn_model = neigh.fit(X_scaled_train, y_train)
getModelStats(knn_model, X_scaled_train, y_train)

{-2: 552, 0: 494, 1: 488, 2: 444, -1: 497}
The accuracy of our model on the data is:  0.8214141414141414
The precision of our model on the data is:  0.8207494721021243
The recall of our model on the data is:  0.8227135471451765
The f1 score of our model on the data is:  0.8208066500916592


In [9]:
# y = employees[metric]
# X = employees.drop([metric], axis=1)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)
# One-hotting the dataset for use
features = ["Race", "Gender", "Birth Origin", "Criminal Record",
           "Arrest Record", "LinkedIn Score", "Responsible Social Media Use", 
            "HireVue Score", "Undergraduate Degree", "Employee Referral"]
prefixes = ["race", "gender", "origin", "criminal", "arrest", "linkedin", 
           "socmedia", "hirevue", "degree", "referral"]
employees_race = employees["Race"]
employees_processed = getOnehottedDataset(employees, features, prefixes)
employees_processed["Race"] = employees_race
employees_processed.columns

metric = "Job Tenure"


# selectedFeatures = [
# #                     "culturalfit_0.0", "culturalfit_1.0", "culturalfit_2.0",
#                     "referral_0.0", "referral_1.0",
#                     "hirevue_0.0", "hirevue_1.0", "hirevue_2.0",
# #                     "education_0.0", "education_1.0", "education_2.0",
# #                     "prestige_0.0", "prestige_1.0", "prestige_2.0",
#                     "linkedin_None", "linkedin_Ok", "linkedin_Very Good",
#                     "socmedia_Bad", "socmedia_Good", "criminal_0.0", "criminal_1.0",
#                     "arrest_0.0", "arrest_1.0",
#                     "GPA", "Education", "Soft Skills", "Leadership Capability", "Educational Prestige"
#                     ]


selectedFeatures = [
    "Military Tenure", "", "Cultural Fit"
]


employees_processed = assignLabels(employees_processed, metric)

In [10]:
# There needs to be a way to examine the labels e.g. 
# Is there consistency in valuations among the features for the high performing ppl?

def printStats(employees, label, features):
    df = employees.copy()
    
    label_vals = df[label].unique()
    label_vals.sort()
    print(label_vals)
    
    for feature in features:
        for label_val in label_vals:
            selectedFeature = df.loc[(df[label] == label_val)][feature]
            print("The mean of " + feature + " for employees with " + str(label_val) + " is", selectedFeature.mean())
            print("The median of " + feature + " for employees " + str(label_val) + " is", selectedFeature.median())
            print("")

printStats(employees_processed, "Label", ["Military Tenure", "Avg Commute Time", "Cultural Fit"])

[-2 -1  0  1  2]
The mean of Military Tenure for employees with -2 is 2.271969696969697
The median of Military Tenure for employees -2 is 2.0

The mean of Military Tenure for employees with -1 is 2.1083333333333334
The median of Military Tenure for employees -1 is 2.0

The mean of Military Tenure for employees with 0 is 1.9636363636363636
The median of Military Tenure for employees 0 is 2.0

The mean of Military Tenure for employees with 1 is 1.9204545454545454
The median of Military Tenure for employees 1 is 2.0

The mean of Military Tenure for employees with 2 is 1.7916666666666667
The median of Military Tenure for employees 2 is 2.0

The mean of Avg Commute Time for employees with -2 is 59.54281902600617
The median of Avg Commute Time for employees -2 is 57.339555758992475

The mean of Avg Commute Time for employees with -1 is 38.4889974108381
The median of Avg Commute Time for employees -1 is 37.905680903764846

The mean of Avg Commute Time for employees with 0 is 24.13686062260133

In [11]:



# employees_processed["Label"] = discretize(employees_processed[metric] / 1, 2)
print(employees_processed.groupby(["Label"]).size())


y = employees_processed["Label"]
X = employees_processed.drop(["Label"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)
X_train_selectedFeatures = X_train[selectedFeatures].copy()
X_test_selectedFeatures = X_test[selectedFeatures].copy()


# Now they get access to a new dataset that has sort of different distributions, they don't know
# Task use kNearestNeighbor or LogisticRegression or something else to try and maximize...
def rescaling(X):
#     print(X)
    scaler = MinMaxScaler()
    scaler.fit(X)
    data = scaler.transform(X)
    return data

X_scaled_train = rescaling(X_train_selectedFeatures)
# print(X_scaled_train)

neigh = KNeighborsClassifier(n_neighbors=4, metric="minkowski", p=3)
knn_model = neigh.fit(X_scaled_train, y_train)


getModelStats(knn_model, X_scaled_train, y_train)

Label
-2    660
-1    660
 0    660
 1    660
 2    660
dtype: int64
{-1: 481, 0: 537, 1: 477, -2: 524, 2: 456}
The accuracy of our model on the data is:  0.8682828282828283
The precision of our model on the data is:  0.8686075767847449
The recall of our model on the data is:  0.8694739548291848
The f1 score of our model on the data is:  0.8684062366073665
